<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_daily_storytelling_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install achatbot

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [2]:
%cd /content/achatbot


/content/achatbot


In [24]:
!bash scripts/pypi_achatbot.sh dev


In [25]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[fastapi_bot_server,ngrok_proxy,daily_room_audio_stream,silero_vad_analyzer,sense_voice_asr,deep_translator,openai,google_ai,together_ai,tts_edge,queue]"


# init api key

run bot task woker with bot.json, e.g.: dummy_bot.json

- use daily/livekit room stream, u can click bot joined the room url, to start chat with bot with audio and camera stream,
  - [daily](https://www.daily.co/) need DAILY_API_KEY
- use openai/groq/together.ai api llm model need api key
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [groq](https://groq.com/) GROQ_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
- use image gen model:
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
  - [huggingface api](https://huggingface.co/docs/api-inference/tasks/text-to-image) HF_API_KEY
  - [HFStableDiffusion](https://huggingface.co/docs/diffusers/using-diffusers/conditional_image_generation) HF_TOKEN/HF_API_KEY




In [9]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
HF_TOKEN=userdata.get('HF_TOKEN')


# download model

In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False


# run bot with config

In [12]:
!cat /content/daily_storytelling_bot.json

{
  "chat_bot_name": "DailyStoryTellingBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "llm": "together",
    "gen_img": "together",
    "tts": "edge"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "llm": {
      "tag": "openai_llm_processor",
      "base_url": "https://api.together.xyz/v1",
      "model": "Qwen/Qwen2.5-72B-Instruct-Turbo",
      "language": "zh-CN",
      "messages": []
    },
    "img_gen": {
      "tag": "TogetherImageGenProcessor",
      "args": {
        "gen_rate_s": 10,
        "model": "black-forest-labs/FLUX.1-schnell-Fr

In [13]:
!DAILY_API_KEY=$DAILY_API_KEY TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_storytelling_bot.json

{
    "chat_bot_name": "DailyStoryTellingBot",
    "config": {
        "asr": {
            "args": {
                "language": "zn",
                "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
            },
            "tag": "sense_voice_asr"
        },
        "img_gen": {
            "args": {
                "gen_rate_s": 10,
                "model": "black-forest-labs/FLUX.1-schnell-Free"
            },
            "tag": "TogetherImageGenProcessor"
        },
        "llm": {
            "base_url": "https://api.together.xyz/v1",
            "language": "zh-CN",
            "messages": [],
            "model": "Qwen/Qwen2.5-72B-Instruct-Turbo",
            "tag": "openai_llm_processor"
        },
        "tts": {
            "args": {
                "gender": "Male",
                "language": "zh-CN",
                "voice_name": "zh-CN-YunjianNeural"
            },
            "tag": "tts_edge"
        },
        "vad": {
            "args": {
          

# http fastapi

ui: https://github.com/ai-bot-pro/nextjs-react-web-storytelling

## use colab kernel proxy
just use in google chrome,
need cookie

In [7]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://uvjh72nxy8r-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [ ]:
# https://m08wf4l3jc-496ff2e9c6d22116-8000-colab.googleusercontent.com/docs
# NOTE: just use in google chrome
# for http api server on colab
!DAILY_API_KEY=$DAILY_API_KEY TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.http.server.fastapi_daily_bot_serve --port 8000


## use ngrok proxy

In [18]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')

In [19]:
!ngrok config add-authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [27]:
!DAILY_API_KEY=$DAILY_API_KEY TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.http.server.fastapi_daily_bot_serve --port 8000 --ngrok 1

2024-11-02 17:02:15,435 - pyngrok.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/ngrok.py:283 - connect - Opening tunnel named: http-8000-da4e4542-5d1c-48b1-bb3b-2a4bfb1dcd62
2024-11-02 17:02:15,459 - pyngrok.process.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/process.py:99 - _log_line - t=2024-11-02T17:02:15+0000 lvl=info msg="no configuration paths supplied"
2024-11-02 17:02:15,459 - pyngrok.process.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/process.py:99 - _log_line - t=2024-11-02T17:02:15+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
2024-11-02 17:02:15,459 - pyngrok.process.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/process.py:99 - _log_line - t=2024-11-02T17:02:15+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
2024-11-02 17:02:15,471 - pyngrok.process.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/process.p

# bot task queue redis

In [ ]:
from google.colab import userdata
REDIS_PASSWORD=userdata.get('REDIS_PASSWORD')


In [ ]:
!LOG_LEVEL=debug REDIS_PASSWORD=$REDIS_PASSWORD python -m src.cmd.bots.main -f /content/task_bot.json